In [90]:
# Notebook currently does the following:
# 1. Loads data from a json-file
# 2. Balances counts of negative and positive reviews, then saves the data to csv as partitions

# To implement in transformation:
# 1. TF-IDF of text
# 2. Further cleaning: removing unwanted characters and symbols
# 3. Feature engineering: 
#    - length of text
# 4. Transform "unixReviewTime" to datetime format

In [ ]:
import pandas as pd
import numpy as np
import pathlib

np.random.seed(0)

In [ ]:
CHUNKS_TO_LOAD = 1
CHUNKSIZE = 10000
DATA_PATH = r"C:\Users\AH\new_folder\graduation_project\data\raw\amazon_reviews_books_5.json.gz"

In [ ]:
# Load amazon fashion reviews dataset
reader = pd.read_json(
    DATA_PATH,
    lines=True, 
    compression="gzip",
    chunksize=CHUNKSIZE,
)

features = ["overall", "reviewText", "summary", "unixReviewTime", "reviewerID", "reviewerName"]

In [89]:
for chunk_n in range(1, CHUNKS_TO_LOAD+1):
    # Remove redundant columns
    df = reader.__next__().loc[:, features]

    # Fix imbalance between scores of 4, 5 and 2, 1.
    value_counts = df["overall"].value_counts()
    count_positive = value_counts[5] + value_counts[4]
    count_neg = value_counts[2] + value_counts[1] 
    rows_to_drop = count_positive - count_neg
    
    # Equally many negative as positive samples
    df_balanced = df.drop(
        axis=1, 
        index=np.random.choice(
            df.query("overall == 4 or overall == 5").index, 
            rows_to_drop, 
            replace=False
        )
    )   

    # Extract old filename to construct new name
    path = pathlib.Path(DATA_PATH)

    file_name = path.parts[-1].split(".")[0] + f"_partition_{chunk_n}"
    
    df.to_csv(fr"C:\Users\AH\new_folder\graduation_project\data\transformed\{file_name}.csv")
